In [73]:
import pandas as pd
import numpy as np

from tkinter import *
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords

In [74]:
df = pd.read_csv("https://raw.githubusercontent.com/revanfz/revanfz/main/dataset_lirik.csv", encoding="utf-8")

In [75]:
df["lirik"] = df["lirik"].replace(r'\n', ' ', regex=True).str.casefold()

In [ ]:
df

Menghilangkan Stopwords
```
Stopwords adalah kata yang tidak memiliki makna ketika berdiri sendiri
```

In [77]:
id_stop_factory = stopwords.words('indonesian')
en_stop_factory = stopwords.words('english')

In [78]:
def remove_stop_word_id(text):
    clean_words = []
    text = text.split()
    for word in text:
        if word not in id_stop_factory:
            clean_words.append(word)
    return ' '.join(clean_words)


In [79]:
def remove_stop_word_en(text):
    clean_words = []
    text = text.split()
    for word in text:
        if word not in en_stop_factory:
            clean_words.append(word)
    return ' '.join(clean_words)


In [ ]:
df["lirik"].apply(remove_stop_word_id)
df["lirik"].apply(remove_stop_word_en)

In [81]:
tfidf_v = TfidfVectorizer()
vektor_lirik = tfidf_v.fit_transform(df["lirik"].tolist())

In [197]:
def cari_lagu(input_text, window):
    query = input_text.get().lower().split(' ')
    clean_query = ""
    for i in query:
        if i not in id_stop_factory and i not in en_stop_factory:
            clean_query += i + " "
        else:
            pass

    clean_query = clean_query.strip()
    print(clean_query)
        
    query_vektor = tfidf_v.transform([clean_query])
    cos_sim = cosine_similarity(query_vektor, vektor_lirik)
    sorted_index = np.argsort(cos_sim)[0][::-1]
    
    global index_relevan
    index_relevan = []
    for i in sorted_index:
    # Memfilter hasil dokumen yang memiliki nilai cosine similarity > 0
        if cos_sim[0][i] > 0:
            index_relevan.append([i, cos_sim[0][i]])
            
    for i in index_relevan:
        print("Dokumen {} memiliki cosine similarity sebesar {:.2f}".format(i[0], i[1]))
        
    print(f"Dokumen relevan yang dihasilkan sistem yakni : {len(index_relevan)} ")

    global lagu
    lagu = []
    for i in index_relevan:
        x = df.iloc[i[0]]['judul']
        y = df.iloc[i[0]]['artis']
        z = df.iloc[i[0]]['lirik']
        lagu.append([x, y, z])

    if len(lagu) > 0:
        target = lagu[0][0]
    else:
        target = "Lagu tidak ditemukan."

    hasil = Frame(window)
    hasil.grid(row=0, column=0, sticky='news')
    hasil.tkraise()
    Label(hasil, text="Lagu yang anda cari adalah: ",  font=("Times",20,"bold")).pack()
    
    count = 1
    for i in lagu:
        if count == 2:
            Label(hasil, text="atau mungkin: ").pack()
        Label(hasil, text=f"{count}.\t\t{i[0]}").pack()
        Label(hasil, text=f"oleh {i[1]}").pack()
        count += 1

    Button(hasil, text="search again", command=lambda:hasil.destroy()).pack()
    Button(hasil, text="close", command=lambda:window.destroy()).pack()

In [198]:
window = Tk()
input_text = StringVar()
window.geometry("480x480")
window.title("Song Finder")

homepage = Frame(window)
homepage.grid(row=0, column=0, sticky='news')
Label(homepage, text="Song Finder", bg="teal", fg="white").pack()
Entry(homepage, text="Masukkan penggalan lirik", textvariable=input_text).pack()
Button(homepage, text="Cari", command=lambda:cari_lagu(input_text, window)).pack()

homepage.tkraise()
window.mainloop()

time
Dokumen 14 memiliki cosine similarity sebesar 0.03
Dokumen 3 memiliki cosine similarity sebesar 0.03
Dokumen 0 memiliki cosine similarity sebesar 0.02
Dokumen relevan yang dihasilkan sistem yakni : 3 
rinduku
Dokumen 12 memiliki cosine similarity sebesar 0.05
Dokumen relevan yang dihasilkan sistem yakni : 1 


In [202]:
dokumen_relevan_sebenarnya = {'all the time': [0], 'rasa rinduku': [12]}

In [200]:
print(dokumen_relevan_sebenarnya.values())

dict_values([[0], [9, 12]])


In [203]:

dokumen_relevan_sebenarnya_yang_dihasilkan = 0
dokumen_ke = 0
precision = 0
for i in index_relevan:
    if i[0] in dokumen_relevan_sebenarnya[input_text.get()]:
        dokumen_relevan_sebenarnya_yang_dihasilkan += 1
    dokumen_ke += 1
    precision += dokumen_relevan_sebenarnya_yang_dihasilkan/dokumen_ke

if dokumen_relevan_sebenarnya_yang_dihasilkan > 0:
    recall = dokumen_relevan_sebenarnya_yang_dihasilkan/len(dokumen_relevan_sebenarnya[input_text.get()])
else:
    recall = 0  


print(f"Kueri \"{input_text.get()}\" memiliki nilai:")
print(f"Recall: {str(recall*100).rstrip('0').rstrip('.')} %")
print(f"dan Precision: {str(precision*100).rstrip('0').rstrip('.')} %")

Kueri "rasa rinduku" memiliki nilai:
Recall: 100 %
dan Precision: 100 %
